In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_DATA_DISABLE_DATA_SERVICE"] = "1"


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
traind=pd.read_csv(r"/kaggle/input/spaceship-titanic/train.csv")
testd=pd.read_csv(r"/kaggle/input/spaceship-titanic/test.csv")

In [ ]:
traind.head(5)

In [ ]:
testd.head(5)

In [ ]:
traind.shape

In [ ]:
testd.shape

In [ ]:
traind.info()

In [ ]:
traind.isna().sum()

In [ ]:
for column in traind:
    print(f'{column}:{traind[column].unique()}')

In [ ]:
traind.replace(True,1,inplace=True)
traind.replace(False,0,inplace=True)

In [ ]:
traind.head(5)


In [ ]:
traind=pd.get_dummies(traind,columns=['HomePlanet','Destination'])

In [ ]:
traind.head(5)

In [ ]:
traind.replace(True,1,inplace=True)
traind.replace(False,0,inplace=True)
testd.replace(True,1,inplace=True)
testd.replace(False,0,inplace=True)

In [ ]:
traind.head(5)

In [ ]:
traind[['deck','num','side']]=traind['Cabin'].str.split('/',expand=True)
traind.drop('Cabin',axis='columns',inplace=True)
testd[['deck','num','side']]=testd['Cabin'].str.split('/',expand=True)
testd.drop('Cabin',axis='columns',inplace=True)

In [ ]:
traind.drop('Name',axis=1,inplace=True)
testd.drop('Name',axis=1,inplace=True)

In [ ]:
traind=pd.get_dummies(traind,columns=['deck','side'])
testd=pd.get_dummies(testd,columns=['deck','side'])

In [ ]:
traind.replace(True,1,inplace=True)
traind.replace(False,0,inplace=True)
testd.replace(True,1,inplace=True)
testd.replace(False,0,inplace=True)

In [ ]:
traind

In [ ]:
dropfromtest=['PassengerId']
testd.drop(dropfromtest,axis=1,inplace=True)

In [ ]:
testd=pd.get_dummies(testd,columns=['HomePlanet','Destination'])

In [ ]:
testd.replace(True,1,inplace=True)
testd.replace(False,0,inplace=True)

In [ ]:
testd

In [ ]:
testd.columns

In [ ]:
testd.dtypes

In [ ]:
testd.isna().sum()

In [ ]:
traind.isna().sum()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()


In [ ]:
traind.columns

In [ ]:
traind['HomePlanet_Mars'].unique()

In [ ]:
for col in traind:
    if traind[col].isnull().sum()>1:
        print(f'{col}: {traind[col].dtype}:{traind[col].unique()}')

In [ ]:
traind.dtypes

In [ ]:
traind['num'] = pd.to_numeric(traind['num'], errors='coerce')
testd['num'] = pd.to_numeric(testd['num'], errors='coerce')

In [ ]:
for col in ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','num']:
    traind[col].fillna(traind[col].median(),inplace=True)

for col in ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','num']:
    testd[col].fillna(testd[col].median(),inplace=True)


In [ ]:
for col in ['CryoSleep','VIP']:
    traind[col].fillna(traind[col].mode()[0],inplace=True)

for col in ['CryoSleep','VIP']:
    testd[col].fillna(testd[col].mode()[0],inplace=True)

In [ ]:
traind.isna().sum()

In [ ]:
testd.isnull().sum()

In [ ]:
traind.shape


In [ ]:
testd.shape

In [ ]:
from sklearn.model_selection import train_test_split

x=traind.drop(columns='Transported',axis=1)
y=traind['Transported']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=45,stratify=y)

In [ ]:
x

In [ ]:
traind.shape

In [ ]:
xtrain=xtrain.drop('PassengerId',axis=1)
xtest=xtest.drop('PassengerId',axis=1)

In [ ]:
xtrain.isnull().sum()

In [ ]:
xtest.isnull().sum()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
xtrain=scaler.fit_transform(xtrain)
x_test=scaler.transform(xtest)

In [ ]:
model=keras.Sequential([
    keras.layers.Input(shape=(25,)),
    keras.layers.Dense(20,activation='relu'),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


model.fit(xtrain, ytrain, epochs=50, batch_size=50)


In [ ]:
yp=model.predict(xtest)
yp

In [ ]:
y_pred=[]
for i in yp:
    if i>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test[['deck','num','side']] = test['Cabin'].str.split('/', expand=True)
test.drop('Cabin', axis=1, inplace=True)

test['num'] = pd.to_numeric(test['num'], errors='coerce')
test = scaler.transform(test)  
y_pred = model.predict(test)
y_pred = (y_pred > 0.5).astype(bool).reshape(-1)


In [ ]:
y_pred[:5]

In [ ]:
output=pd.DataFrame({'True':ytest,'Predicted':y_pred,'match':ytest==y_pred})
output[:5]

In [ ]:
output['match'].value_counts()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(ytest,y_pred))

In [ ]:
submissionformat=pd.read_csv(r"/kaggle/input/spaceship-titanic/sample_submission.csv")
submissionformat

In [ ]:
plt.figure()
cm=tf.math.confusion_matrix(labels=ytest,predictions=y_pred)
sns.heatmap(cm,annot=True,fmt='d')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')

In [ ]:
originaltrain=pd.read_csv(r"/kaggle/input/spaceship-titanic/test.csv")
y_pred = pd.Series(y_pred)
y_pred.replace(1,'True',inplace=True)
y_pred.replace(0,'False',inplace=True)

In [ ]:
y_pred.tail(10)

In [ ]:
len(originaltrain), len(y_pred)


In [ ]:
submission=pd.DataFrame({'PassengerId':originaltrain['PassengerId'],'Transported':y_pred})
submission

In [ ]:
submission.to_csv("submission.csv", index=False)
